In [8]:
!pip install pyarrow


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [9]:
import numpy as np
import pandas as pd
import os
import sys
import pyarrow.parquet as pq
import re
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pytz
import math
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize

In [10]:
month_dict = { 1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
folder_path = r'/data/workspace_files/parquet_files'

In [52]:
def get_date_str_for_hp(start_year, end_year):
    date_strs = []
    for yr in range(start_year, end_year + 1):
        for month in range(1,13):
            if yr == 1996 and month <= 4:
                continue
            date_strs.append(str(month) + str(yr))
    return date_strs

In [53]:
def get_monthly_returns(returns, hold_period):
    return (1+ret)^(1/hold_period) - 1

In [54]:
def get_long_or_short_side(portfolio, pos_type):
    portfolio = portfolio.copy(deep = True)
    side = portfolio[portfolio['pos_type'] == pos_type].copy(deep = True)
    weights = np.array(side['weight']).reshape(-1,)*pos_type
    ret = np.array(side['ret']).reshape(-1,)
    ret_clipped = np.array(side['ret_clipped']).reshape(-1,)
    ret_clipped2 = np.array(side['ret_clipped2']).reshape(-1,)
    total_return = (weights*ret).sum()
    total_return_clipped = (weights*ret_clipped).sum()
    total_return_clipped2 = (weights*ret_clipped2).sum()

    return total_return, total_return_clipped, total_return_clipped2

In [55]:
def compute_portfolio_returns(score_type, hold_period, history, n_bucket, date_strs, quartile = None):
    tot_returns = []
    folder_path_req = f'{folder_path}/F_and_H/f{hold_period}_and_h{history}/n_{n_bucket}/{score_type}'
    for date_str_idx in range(0, len(date_strs), hold_period):
            date_str = date_strs[date_str_idx]
            date_obj = datetime.strptime(date_str, '%m%Y')
            date_for_files = date_obj.strftime('%b%Y')
            if quartile != None:
                date_for_files = f'Q{quartile}/{date_for_files}'
            else:
                date_for_files = f'LS/{date_for_files}'
            score_path = f'{folder_path_req}/{date_for_files}.parquet'
            print(score_path)
            portfolio = pq.read_table(score_path).to_pandas().dropna(subset=['weight'])
            #print(portfolio)
            portfolio['ret_clipped2'] = winsorize(portfolio['ret'], limits=[0.01, 0.01])
            long_ret, long_ret_clipped, long_ret_clipped2 = get_long_or_short_side(portfolio, 1)
            short_ret, short_ret_clipped, short_ret_clipped2 = get_long_or_short_side(portfolio, -1)
            # for time_stamp_idx in range(hold_period):
            #     date_obj_req = date_obj + relativedelta(months = time_stamp_idx)
            tot_returns.append({'date': date_obj, 'return': long_ret - short_ret, 'return_clipped': long_ret_clipped - short_ret_clipped, 'return_clipped2': long_ret_clipped2 - short_ret_clipped2, 'num_holdings': len(portfolio),
                                    'long_return': long_ret, 'long_return_clipped': long_ret_clipped, 'long_return_clipped2': long_ret_clipped2,
                                    'short_return': short_ret, 'short_return_clipped': short_ret_clipped, 'short_return_clipped2': short_ret_clipped2})

    ret_df = pd.DataFrame(tot_returns)
    return ret_df

In [57]:
date_strs

['51996',
 '61996',
 '71996',
 '81996',
 '91996',
 '101996',
 '111996',
 '121996',
 '11997',
 '21997',
 '31997',
 '41997',
 '51997',
 '61997',
 '71997',
 '81997',
 '91997',
 '101997',
 '111997',
 '121997',
 '11998',
 '21998',
 '31998',
 '41998',
 '51998',
 '61998',
 '71998',
 '81998',
 '91998',
 '101998',
 '111998',
 '121998',
 '11999',
 '21999',
 '31999',
 '41999',
 '51999',
 '61999',
 '71999',
 '81999',
 '91999',
 '101999',
 '111999',
 '121999',
 '12000',
 '22000',
 '32000',
 '42000',
 '52000',
 '62000',
 '72000',
 '82000',
 '92000',
 '102000',
 '112000',
 '122000',
 '12001',
 '22001',
 '32001',
 '42001',
 '52001',
 '62001',
 '72001',
 '82001',
 '92001',
 '102001',
 '112001',
 '122001',
 '12002',
 '22002',
 '32002',
 '42002',
 '52002',
 '62002',
 '72002',
 '82002',
 '92002',
 '102002',
 '112002',
 '122002',
 '12003',
 '22003',
 '32003',
 '42003',
 '52003',
 '62003',
 '72003',
 '82003',
 '92003',
 '102003',
 '112003',
 '122003',
 '12004',
 '22004',
 '32004',
 '42004',
 '52004',
 '6200

In [56]:
date_strs = get_date_str_for_hp(1996, 2018)
for score_type in ['cosine_similarity', 'jaccard_similarity']:
    for hold_period in [1,3,6]:
        for history in [1,3,6]:
            if history < hold_period:
                    continue
            for num_bucket in [5,10]:
                rets = compute_portfolio_returns(score_type, hold_period, history, num_bucket, date_strs)
                file_path_req = f'{folder_path}/F_and_H/f{hold_period}_and_h{history}/n_{num_bucket}/{score_type}/ret.parquet'
                rets.to_parquet(file_path_req)

/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/May1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Jun1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Jul1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Aug1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Sep1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Oct1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Nov1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Dec1996.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Jan1997.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and_h1/n_5/cosine_similarity/LS/Feb1997.parquet
/data/workspace_files/parquet_files/F_and_H/f1_and